# 1. Import packages

In [1]:
import copy
import ipaddress
import math
import numpy as np
import os
import pandas as pd
import re
import time

from datetime import timedelta

# 2. Read a sample CSV

## 2.1. Change to inputs directory

In [2]:
print(os.getcwd())
input_path_s = "/input/"
input_path_s = os.getcwd() + input_path_s
output_path_s = "/output/"
output_path_s = os.getcwd() + output_path_s
os.chdir(input_path_s)
print(os.getcwd())

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain
/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain/input


## 2.2. Read CSV: 2022-08-09.csv

In [3]:
bright_data_df = pd.read_csv("domains_bright_data.csv")

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
bright_data_df.iloc[20:30]

,country,asn,initial_domain_filename,target_domain
20,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,www.google.hr
21,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,apis.google.com
22,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,ogs.google.hr
23,hr,61094,20220804-HR-61094-httpswww.njuskalo.hr-DNS_Req...,ls.hit.gemius.pl
24,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,aax-eu.amazon-adsystem.com
25,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,buy.tinypass.com
26,hr,61094,20220804-HR-61094-httpswww.roblox.com-DNS_Requ...,static.rbxcdn.com
27,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,cdn.ampproject.org
28,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,ads.yahoo.com
29,hr,61094,20220804-HR-61094-httpswww.srce.hr-DNS_Request...,www.srce.unizg.hr


In [5]:
bright_data_df.shape

(1075319, 4)

# 3. Column 5 and 6.

In [6]:
original_bright_data_df = copy.deepcopy(bright_data_df)

In [7]:
# I am aware that looping through a dataframe is not the best way.
# However, a data maninuplation onto a million row of data leads Python crash.
# Please, let me know how I can improve my code. Thanks.

## 3.1. Create TLD + 1

In [8]:
initial_domain_tld_plus_one_l = []
target_domain_tld_plus_one_l = []


for i in range(len(bright_data_df)):
    if bright_data_df["asn"][i] == "google":
        initial_domain_tld_plus_one_l.append(None)
        target_domain_tld_plus_one_l.append(None)
    else:
        # I am pretty sure there is a better way to regex once, but I am doing regex twice.
        initial_domain_tld_plus_one_s = re.search("\.(.*)-DNS_Requests.txt", bright_data_df["initial_domain_filename"][i]).group(1)
        initial_domain_tld_plus_one_s = re.search("(\w+\.\w+)$", initial_domain_tld_plus_one_s).group(1)
        initial_domain_tld_plus_one_l.append(initial_domain_tld_plus_one_s)
        target_domain_tld_plus_one_s = re.search("(\w+\.\w+)$", bright_data_df["target_domain"][i]).group(1)
        target_domain_tld_plus_one_l.append(target_domain_tld_plus_one_s)

## 3.2. Check whether there is only one dot or not

In [9]:
# It is good if nothing is printed.
for tld_plus_one in initial_domain_tld_plus_one_l:
    try:
        dot_count_i = tld_plus_one.count(".")
        if dot_count_i > 1:
            print(tld_plus_one)
    except:
        pass

In [10]:
# It is good if nothing is printed.
for tld_plus_one in target_domain_tld_plus_one_l:
    try:
        dot_count_i = tld_plus_one.count(".")
        if dot_count_i > 1:
            print(tld_plus_one)
    except:
        pass

## 3.3. Add new columns

In [11]:
bright_data_df["initial_domain_tld_plus_one"] = initial_domain_tld_plus_one_l
bright_data_df["target_domain_tld_plus_one"] = target_domain_tld_plus_one_l

In [12]:
bright_data_df.iloc[20:30]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one
20,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,www.google.hr,None,None
21,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,apis.google.com,None,None
22,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,ogs.google.hr,None,None
23,hr,61094,20220804-HR-61094-httpswww.njuskalo.hr-DNS_Req...,ls.hit.gemius.pl,njuskalo.hr,gemius.pl
24,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,aax-eu.amazon-adsystem.com,24sata.hr,adsystem.com
25,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,buy.tinypass.com,24sata.hr,tinypass.com
26,hr,61094,20220804-HR-61094-httpswww.roblox.com-DNS_Requ...,static.rbxcdn.com,roblox.com,rbxcdn.com
27,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,cdn.ampproject.org,gloria.hr,ampproject.org
28,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,ads.yahoo.com,24sata.hr,yahoo.com
29,hr,61094,20220804-HR-61094-httpswww.srce.hr-DNS_Request...,www.srce.unizg.hr,srce.hr,unizg.hr


In [13]:
bright_data_df.shape

(1075319, 6)

# 4. Internet Measurement Tools

## 4.1. dig: (url to) hostname to ip

In [14]:
initial_domain_hostname_set = set(bright_data_df["initial_domain_tld_plus_one"])
target_domain_hostname_set = set(bright_data_df["target_domain_tld_plus_one"])

In [80]:
initial_domain_hostname_set == target_domain_hostname_set

False

In [15]:
start_time = time.time()
initial_domain_hostname_ip_d = {}

for hostname_s in initial_domain_hostname_set:
    initial_domain_hostname_ip_d[hostname_s] = []

    if hostname_s is not None:
        cmd = "dig +short " + hostname_s
        ip_l = os.popen(cmd).read().split("\n")

        for ip in ip_l:
            if ip != "":
                try:
                    ipaddress.ip_address(ip)
                    initial_domain_hostname_ip_d[hostname_s].append(ip)
                except ValueError:
                    print(f"{hostname_s} has an invalid address/netmask at {ip}")
                except:
                    print("Usage : %s  ip" % ip)
                

elapsed_time = time.time() - start_time
print(str(timedelta(seconds=elapsed_time)))
print(list(initial_domain_hostname_ip_d.items())[:10])

0:01:48.862133
[('kuleuven.be', ['134.58.64.15']), ('mundodeportivo.com', ['51.68.114.63', '51.77.228.215']), ('pravda.sk', ['217.67.31.48']), ('meteo.be', ['193.190.231.166']), ('futbol24.com', ['213.227.146.165', '213.227.146.161', '213.227.146.163']), ('makeleio.gr', ['62.38.6.97']), ('voici.fr', ['54.73.86.40']), ('marica.bg', ['172.67.73.120', '104.26.4.228', '104.26.5.228']), ('sverigesradio.se', ['134.25.4.140']), ('femina.hu', ['194.143.245.39', '213.181.199.14', '213.181.199.15', '213.181.199.16', '213.181.199.17', '81.0.120.157', '81.0.120.158'])]


In [16]:
start_time = time.time()
target_domain_hostname_ip_d = {}

for hostname_s in target_domain_hostname_set:
    target_domain_hostname_ip_d[hostname_s] = []

    if hostname_s is not None:
        cmd = "dig +short " + hostname_s
        ip_l = os.popen(cmd).read().split("\n")

        for ip in ip_l:
            if ip != "":
                try:
                    ipaddress.ip_address(ip)
                    target_domain_hostname_ip_d[hostname_s].append(ip)
                except ValueError:
                    print(f"{hostname_s} has an invalid address/netmask at {ip}")
                except:
                    print("Usage : %s  ip" % ip)
                

elapsed_time = time.time() - start_time
print(str(timedelta(seconds=elapsed_time)))
print(list(target_domain_hostname_ip_d.items())[:10])

0:04:34.105650
[('kuleuven.be', ['134.58.64.15']), ('mundodeportivo.com', ['51.77.228.215', '51.68.114.63']), ('onetrust.io', []), ('ingatlantajolo.hu', ['54.75.179.111', '52.51.212.20']), ('rmbl.ws', ['151.139.128.10']), ('advangelists.com', ['166.62.107.20']), ('metweb.ie', ['172.105.245.48']), ('pdst.fm', ['216.239.38.21', '216.239.32.21', '216.239.34.21', '216.239.36.21']), ('radiantmediatechs.com', ['172.67.8.9', '104.22.62.150', '104.22.63.150']), ('voici.fr', ['54.73.86.40'])]


## 4.2. whois: ip to AS Number (ASN) and AS Name (AS)

In [50]:
print(len(initial_domain_hostname_set) == len(initial_domain_hostname_ip_d.keys()))
print(len(target_domain_hostname_set) == len(target_domain_hostname_ip_d.keys()))

True
True


In [53]:
start_time = time.time()
initial_domain_hostname_asn_d = {}
initial_domain_hostname_as_d = {}

for hostname, ip_l in initial_domain_hostname_ip_d.items():

    # Sanity check.
    # If ever the length of ASN and AS do not match, then break this loop
    if len(initial_domain_hostname_asn_d.values()) != len(initial_domain_hostname_as_d.values()):
        print(hostname, ip)
        break

    initial_domain_hostname_asn_d[hostname] = []
    initial_domain_hostname_as_d[hostname] = []
    
    for ip in ip_l:
        cmd = f"whois -h whois.cymru.com {ip}"
        # try:
        cymru_l = os.popen(cmd).read().split("\n")[1].split("|")

        for index, item in enumerate(cymru_l):
            if index == 0:
                initial_domain_hostname_asn_d[hostname].append(item.strip())
            if index == 2:
                initial_domain_hostname_as_d[hostname].append(item.strip())
        # except:
        #     cymru_l = os.popen(cmd).read()
        #     print(cymru_l)
        #     break



elapsed_time = time.time() - start_time
print(str(timedelta(seconds=elapsed_time)))
print(list(initial_domain_hostname_asn_d.items())[:10])
print(list(initial_domain_hostname_as_d.items())[:10])

0:03:04.862925
[('kuleuven.be', ['2611']), ('mundodeportivo.com', ['16276', '16276']), ('pravda.sk', ['5578']), ('meteo.be', ['2611']), ('futbol24.com', ['60781', '60781', '60781']), ('makeleio.gr', ['3329']), ('voici.fr', ['16509']), ('marica.bg', ['13335', '13335', '13335']), ('sverigesradio.se', ['47708']), ('femina.hu', ['12301', '42371', '42371', '42371', '42371', '12301', '12301'])]
[('kuleuven.be', ['BELNET, BE']), ('mundodeportivo.com', ['OVH, FR', 'OVH, FR']), ('pravda.sk', ['AS-BENESTRA Bratislava, Slovak Republic, SK']), ('meteo.be', ['BELNET, BE']), ('futbol24.com', ['LEASEWEB-NL-AMS-01 Netherlands, NL', 'LEASEWEB-NL-AMS-01 Netherlands, NL', 'LEASEWEB-NL-AMS-01 Netherlands, NL']), ('makeleio.gr', ['HOL-GR Athens, Greece, GR']), ('voici.fr', ['AMAZON-02, US']), ('marica.bg', ['CLOUDFLARENET, US', 'CLOUDFLARENET, US', 'CLOUDFLARENET, US']), ('sverigesradio.se', ['SVERIGES-RADIO Sveriges Radio AB, SE']), ('femina.hu', ['INVITECH, HU', 'AS_MAGEX, HU', 'AS_MAGEX, HU', 'AS_MAGEX,

In [54]:
start_time = time.time()
target_domain_hostname_asn_d = {}
target_domain_hostname_as_d = {}

for hostname, ip_l in target_domain_hostname_ip_d.items():

    # Sanity check.
    # If ever the length of ASN and AS do not match, then break this loop
    if len(target_domain_hostname_asn_d.values()) != len(target_domain_hostname_as_d.values()):
        print(hostname, ip)
        break

    target_domain_hostname_asn_d[hostname] = []
    target_domain_hostname_as_d[hostname] = []
    
    for ip in ip_l:
        cmd = f"whois -h whois.cymru.com {ip}"
        # try:
        cymru_l = os.popen(cmd).read().split("\n")[1].split("|")

        for index, item in enumerate(cymru_l):
            if index == 0:
                target_domain_hostname_asn_d[hostname].append(item.strip())
            if index == 2:
                target_domain_hostname_as_d[hostname].append(item.strip())
        # except:
        #     cymru_l = os.popen(cmd).read()
        #     print(cymru_l)
        #     break



elapsed_time = time.time() - start_time
print(str(timedelta(seconds=elapsed_time)))
print(list(target_domain_hostname_asn_d.items())[:10])
print(list(target_domain_hostname_as_d.items())[:10])

0:08:28.462231
[('kuleuven.be', ['2611']), ('mundodeportivo.com', ['16276', '16276']), ('onetrust.io', []), ('ingatlantajolo.hu', ['16509', '16509']), ('rmbl.ws', ['20446']), ('advangelists.com', ['398101']), ('metweb.ie', ['63949']), ('pdst.fm', ['15169', '15169', '15169', '15169']), ('radiantmediatechs.com', ['13335', '13335', '13335']), ('voici.fr', ['16509'])]
[('kuleuven.be', ['BELNET, BE']), ('mundodeportivo.com', ['OVH, FR', 'OVH, FR']), ('onetrust.io', []), ('ingatlantajolo.hu', ['AMAZON-02, US', 'AMAZON-02, US']), ('rmbl.ws', ['STACKPATH-CDN, US']), ('advangelists.com', ['GO-DADDY-COM-LLC, US']), ('metweb.ie', ['LINODE-AP Linode, LLC, US']), ('pdst.fm', ['GOOGLE, US', 'GOOGLE, US', 'GOOGLE, US', 'GOOGLE, US']), ('radiantmediatechs.com', ['CLOUDFLARENET, US', 'CLOUDFLARENET, US', 'CLOUDFLARENET, US']), ('voici.fr', ['AMAZON-02, US'])]


## 4.3. AS Number to AS2Org_OrgID

In [61]:
asn_orgid_d = {}
orgid_orgname_d = {}

with open('./20220701.as-org2info.txt', 'r') as caida_file:
    for line in caida_file.readlines():
        if not line.startswith("#"):
            items = line.strip().split("|")
            if len(items) == 5:
                orgid_orgname_d[items[0]] = items[2]
            elif len(items) == 6:
                asn_orgid_d[int(items[0])] = items[3]
            else:
                assert False
                
print(list(asn_orgid_d.items())[:10])
print()
print(list(orgid_orgname_d.items())[:10])

[(1, 'LPL-141-ARIN'), (2, 'UNIVER-19-ARIN'), (3, 'MIT-2-ARIN'), (4, 'USC-32-ARIN'), (5, 'WGL-117-ARIN'), (6, 'ATOS-Z-ARIN'), (7, 'ORG-TDSA4-RIPE'), (8, 'RICEUN-ARIN'), (9, 'CARNEG-Z-ARIN'), (10, 'CCICC-ARIN')]

[('111S-ARIN', 'One Eleven Internet Services'), ('1800CO-2-ARIN', '1-800 Contacts, Inc.'), ('1800FL-ARIN', '1-800-Flowers.com, Inc.'), ('1800H-ARIN', '1-800-HOSTING, Inc.'), ('1881CS-ARIN', '1881CS/XPC'), ('1FBU-ARIN', '1st Financial Bank USA'), ('1GPUC-ARIN', '1798 Global Partners (USA), Corp.'), ('1STP-ARIN', 'FIRST STEP INTERNET, LLC'), ('1UWEB-ARIN', '1U Web, INC.'), ('280INC-ARIN', '280, Inc.')]


In [63]:
initial_domain_hostname_orgid_d = {}
initial_domain_hostname_orgname_d = {}

for hostname, asn_l in initial_domain_hostname_asn_d.items():

    initial_domain_hostname_orgid_d[hostname] = set()
    initial_domain_hostname_orgname_d[hostname] = set()

    for asn in asn_l:        
        try:
            initial_domain_hostname_orgid_d[hostname].add(asn_orgid_d[int(asn)])
            initial_domain_hostname_orgname_d[hostname].add(orgid_orgname_d[asn_orgid_d[int(asn)]])
        except:
            print(f"AS number {asn} does not exist")
    
print(len(initial_domain_hostname_asn_d.values()) == len(initial_domain_hostname_orgid_d.values()))
print(list(initial_domain_hostname_orgid_d.items())[:10])
print(len(initial_domain_hostname_asn_d.values()) == len(initial_domain_hostname_orgname_d.values()))
print(list(initial_domain_hostname_orgname_d.items())[:10])

AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
True
[('kuleuven.be', {'ORG-BA5-RIPE'}), ('mundodeportivo.com', {'ORG-OS3-RIPE'}), ('pravda.sk', {'ORG-SPA43-RIPE'}), ('meteo.be', {'ORG-BA5-RIPE'}), ('futbol24.com', {'ORG-OB3-RIPE'}), ('makeleio.gr', {'ORG-PS3-RIPE'}), ('voici.fr', {'AMAZON-4-ARIN'}), ('marica.bg', {'CLOUD14-ARIN'}), ('sverigesradio.se', {'ORG-SR18-RIPE'}), ('femina.hu', {'ORG-MSK7-RIPE', 'ORG-DC2-RIPE'})]
True
[('kuleuven.be', {'BELNET'}), ('mundodeportivo.com', {'OVH SAS'}), ('pravda.sk', {'SWAN, a.s.'}), ('meteo.be', {'BELNET'}), ('futbol24.com', {'LeaseWeb Netherlands B.V.'}), ('makeleio.gr', {'VODAFONE-PANAFON HELLENIC TELECOMMUNICATIONS COMPANY SA'}), ('voici.fr', {'Amazon.com, Inc.'}), ('marica.bg', {'Cloudflare, Inc.'}), ('sverigesradio.se', {'Sveriges Radio AB'

In [64]:
target_domain_hostname_orgid_d = {}
target_domain_hostname_orgname_d = {}

for hostname, asn_l in target_domain_hostname_asn_d.items():

    target_domain_hostname_orgid_d[hostname] = set()
    target_domain_hostname_orgname_d[hostname] = set()

    for asn in asn_l:        
        try:
            target_domain_hostname_orgid_d[hostname].add(asn_orgid_d[int(asn)])
            target_domain_hostname_orgname_d[hostname].add(orgid_orgname_d[asn_orgid_d[int(asn)]])
        except:
            print(f"AS number {asn} does not exist")
    
print(len(target_domain_hostname_asn_d.values()) == len(target_domain_hostname_orgid_d.values()))
print(list(target_domain_hostname_orgid_d.items())[:10])
print(len(target_domain_hostname_asn_d.values()) == len(target_domain_hostname_orgname_d.values()))
print(list(target_domain_hostname_orgname_d.items())[:10])

AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number 60922 does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
AS number NA does not exist
True
[('kuleuven.be', {'ORG-BA5-RIPE'}), ('mundodeportivo.com', {'ORG-OS3-RIPE'}), ('onetrust.io', set()), ('ingatlantajolo.hu', {'AMAZON-4-ARIN'}), ('rmbl.ws', {'SL-892-ARIN'}), ('advangelists.com', {'GODAD-ARIN'}), ('metweb.ie', {'ORG-LL3-AP-APNIC'}), ('pdst.fm', {'GOGL-ARIN'}), ('radiantmediatechs.com', {'CLOUD14-ARIN'}), ('voici.fr', {'AMAZON-4-ARIN'})]
True
[('kuleuven.be',

# 5. First Party vs Third Party

## 5.1. Method: TLD+1 matching

In [24]:
print(len(initial_domain_tld_plus_one_l) == len(target_domain_tld_plus_one_l))
print(len(initial_domain_tld_plus_one_l) == bright_data_df.shape[0])

True
True


In [175]:
tld_plus_one_first_party_l = bright_data_df.shape[0] * [None]

for i in range(len(initial_domain_tld_plus_one_l)):
    if initial_domain_tld_plus_one_l[i] is not None and target_domain_tld_plus_one_l[i] is not None and initial_domain_tld_plus_one_l[i] == target_domain_tld_plus_one_l[i]:
        tld_plus_one_first_party_l[i] = True
        
print(len(tld_plus_one_first_party_l) == len(initial_domain_tld_plus_one_l))
print(tld_plus_one_first_party_l.count(True))
print(f"At least {round(tld_plus_one_first_party_l.count(True) / len(tld_plus_one_first_party_l) * 100, 5)} % is 1st party.")
print(tld_plus_one_first_party_l[:100])

True
177982
At least 16.55155 % is 1st party.
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, True, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, True, None, None, True, None, None, None, True, None, None, None, True, None, None, None, True, None, True, None, None, None, None, None, None, None]


## 5.2. Method: AS (AS Name)

In [70]:
match_hostname_as_d = {}

for initial_hostname, initial_as in initial_domain_hostname_as_d.items():
    if target_domain_hostname_as_d[initial_hostname] == initial_as:
        match_hostname_as_d[initial_hostname] = True
        
print(len(match_hostname_as_d) < len(initial_domain_hostname_as_d))
print(list(match_hostname_as_d.items())[:10])

True
[('kuleuven.be', True), ('mundodeportivo.com', True), ('pravda.sk', True), ('meteo.be', True), ('futbol24.com', True), ('makeleio.gr', True), ('voici.fr', True), ('marica.bg', True), ('sverigesradio.se', True), ('mbank.pl', True)]


## 5.3. Method: ASN (AS Number)

In [71]:
match_hostname_asn_d = {}

for initial_hostname, initial_asn in initial_domain_hostname_asn_d.items():
    if target_domain_hostname_asn_d[initial_hostname] == initial_asn:
        match_hostname_asn_d[initial_hostname] = True
        
print(len(match_hostname_asn_d) < len(initial_domain_hostname_asn_d))
print(list(match_hostname_asn_d.items())[:10])

True
[('kuleuven.be', True), ('mundodeportivo.com', True), ('pravda.sk', True), ('meteo.be', True), ('futbol24.com', True), ('makeleio.gr', True), ('voici.fr', True), ('marica.bg', True), ('sverigesradio.se', True), ('mbank.pl', True)]


## 5.4. Method: Org ID

In [76]:
match_hostname_orgid_d = {}

for initial_hostname, initial_orgid in initial_domain_hostname_orgid_d.items():
    if target_domain_hostname_orgid_d[initial_hostname] == initial_orgid:
        match_hostname_orgid_d[initial_hostname] = True
        
print(len(match_hostname_orgid_d) <= len(initial_domain_hostname_orgid_d))
print(list(match_hostname_orgid_d.items())[:10])

True
[('kuleuven.be', True), ('mundodeportivo.com', True), ('pravda.sk', True), ('meteo.be', True), ('futbol24.com', True), ('makeleio.gr', True), ('voici.fr', True), ('marica.bg', True), ('sverigesradio.se', True), ('femina.hu', True)]


## 5.5. Method: Org Name

In [77]:
match_hostname_orgname_d = {}

for initial_hostname, initial_orgname in initial_domain_hostname_orgname_d.items():
    if target_domain_hostname_orgname_d[initial_hostname] == initial_orgname:
        match_hostname_orgname_d[initial_hostname] = True
        
print(len(match_hostname_orgname_d) <= len(initial_domain_hostname_orgname_d))
print(list(match_hostname_orgname_d.items())[:10])

True
[('kuleuven.be', True), ('mundodeportivo.com', True), ('pravda.sk', True), ('meteo.be', True), ('futbol24.com', True), ('makeleio.gr', True), ('voici.fr', True), ('marica.bg', True), ('sverigesradio.se', True), ('femina.hu', True)]


# 6. A combined df: Add columns

## 6.1. tld_plus_one_first_party_l

In [86]:
bright_data_6col_df = copy.deepcopy(bright_data_df)

In [89]:
bright_data_df.iloc[20:30]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one
20,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,www.google.hr,None,None
21,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,apis.google.com,None,None
22,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,ogs.google.hr,None,None
23,hr,61094,20220804-HR-61094-httpswww.njuskalo.hr-DNS_Req...,ls.hit.gemius.pl,njuskalo.hr,gemius.pl
24,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,aax-eu.amazon-adsystem.com,24sata.hr,adsystem.com
25,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,buy.tinypass.com,24sata.hr,tinypass.com
26,hr,61094,20220804-HR-61094-httpswww.roblox.com-DNS_Requ...,static.rbxcdn.com,roblox.com,rbxcdn.com
27,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,cdn.ampproject.org,gloria.hr,ampproject.org
28,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,ads.yahoo.com,24sata.hr,yahoo.com
29,hr,61094,20220804-HR-61094-httpswww.srce.hr-DNS_Request...,www.srce.unizg.hr,srce.hr,unizg.hr


In [176]:
bright_data_df["tld_plus_one_first_party"] = tld_plus_one_first_party_l

In [119]:
bright_data_df.iloc[50:60]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party
50,hr,61094,20220804-HR-61094-httpswww.tportal.hr-DNS_Requ...,www.tportal.hr,tportal.hr,tportal.hr,True
51,hr,61094,20220804-HR-61094-httpswww.poslovni.hr-DNS_Req...,hr.hit.gemius.pl,poslovni.hr,gemius.pl,None
52,hr,61094,20220804-HR-61094-httpswww.net.hr-DNS_Requests...,s.amazon-adsystem.com,net.hr,adsystem.com,None
53,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,www.youtube.com,gloria.hr,youtube.com,None
54,hr,61094,20220804-HR-61094-httpswww.index.hr-DNS_Reques...,script.dotmetrics.net,index.hr,dotmetrics.net,None
55,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,connect.facebook.net,vecernji.hr,facebook.net,None
56,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,cdnjs.cloudflare.com,vecernji.hr,cloudflare.com,None
57,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-beacon.s-onetag.com,jutarnji.hr,onetag.com,None
58,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,apps.jutarnji.hr,gloria.hr,jutarnji.hr,None
59,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-metrics-collector-beta.s-onetag.com,jutarnji.hr,onetag.com,None


## 6.2. as_name_first_party_l

In [120]:
bright_data_7col_df = copy.deepcopy(bright_data_df)

In [147]:
as_name_first_party_l = bright_data_df.shape[0] * [None]
match_as_name_set = set(match_hostname_as_d.keys())
match_as_name_set.remove(None)

for i in range(len(initial_domain_tld_plus_one_l)):
    if initial_domain_tld_plus_one_l[i] in match_as_name_set:
        as_name_first_party_l[i] = True
        
print(as_name_first_party_l.count(None) + as_name_first_party_l.count(True) == bright_data_df.shape[0])
print(f"{round(as_name_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% is the first party based on AS.")

True
98.06457% is the first party based on AS.


In [142]:
bright_data_df["as_name_first_party"] = as_name_first_party_l

In [143]:
bright_data_df.iloc[50:60]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party
50,hr,61094,20220804-HR-61094-httpswww.tportal.hr-DNS_Requ...,www.tportal.hr,tportal.hr,tportal.hr,True,True
51,hr,61094,20220804-HR-61094-httpswww.poslovni.hr-DNS_Req...,hr.hit.gemius.pl,poslovni.hr,gemius.pl,None,True
52,hr,61094,20220804-HR-61094-httpswww.net.hr-DNS_Requests...,s.amazon-adsystem.com,net.hr,adsystem.com,None,True
53,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,www.youtube.com,gloria.hr,youtube.com,None,True
54,hr,61094,20220804-HR-61094-httpswww.index.hr-DNS_Reques...,script.dotmetrics.net,index.hr,dotmetrics.net,None,True
55,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,connect.facebook.net,vecernji.hr,facebook.net,None,True
56,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,cdnjs.cloudflare.com,vecernji.hr,cloudflare.com,None,True
57,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-beacon.s-onetag.com,jutarnji.hr,onetag.com,None,True
58,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,apps.jutarnji.hr,gloria.hr,jutarnji.hr,None,True
59,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-metrics-collector-beta.s-onetag.com,jutarnji.hr,onetag.com,None,True


In [144]:
bright_data_df.shape

(1075319, 8)

## 6.3. as_number_first_party_l

In [145]:
bright_data_8col_df = copy.deepcopy(bright_data_df)

In [154]:
as_number_first_party_l = bright_data_df.shape[0] * [None]
match_as_number_set = set(match_hostname_asn_d.keys())
match_as_number_set.remove(None)

for i in range(len(initial_domain_tld_plus_one_l)):
    if initial_domain_tld_plus_one_l[i] in match_as_number_set:
        as_number_first_party_l[i] = True
        
print(as_number_first_party_l.count(None) + as_number_first_party_l.count(True) == bright_data_df.shape[0])
print(f"{round(as_number_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% is the first party based on ASN.")

True
98.06457% is the first party based on ASN.


In [149]:
bright_data_df["as_number_first_party"] = as_number_first_party_l

In [150]:
bright_data_df.iloc[50:60]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party,as_number_first_party
50,hr,61094,20220804-HR-61094-httpswww.tportal.hr-DNS_Requ...,www.tportal.hr,tportal.hr,tportal.hr,True,True,True
51,hr,61094,20220804-HR-61094-httpswww.poslovni.hr-DNS_Req...,hr.hit.gemius.pl,poslovni.hr,gemius.pl,None,True,True
52,hr,61094,20220804-HR-61094-httpswww.net.hr-DNS_Requests...,s.amazon-adsystem.com,net.hr,adsystem.com,None,True,True
53,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,www.youtube.com,gloria.hr,youtube.com,None,True,True
54,hr,61094,20220804-HR-61094-httpswww.index.hr-DNS_Reques...,script.dotmetrics.net,index.hr,dotmetrics.net,None,True,True
55,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,connect.facebook.net,vecernji.hr,facebook.net,None,True,True
56,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,cdnjs.cloudflare.com,vecernji.hr,cloudflare.com,None,True,True
57,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-beacon.s-onetag.com,jutarnji.hr,onetag.com,None,True,True
58,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,apps.jutarnji.hr,gloria.hr,jutarnji.hr,None,True,True
59,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-metrics-collector-beta.s-onetag.com,jutarnji.hr,onetag.com,None,True,True


In [151]:
bright_data_df.shape

(1075319, 9)

## 6.4. org_id_first_party_l

In [152]:
bright_data_9col_df = copy.deepcopy(bright_data_df)

In [155]:
org_id_first_party_l = bright_data_df.shape[0] * [None]
match_orgid_set = set(match_hostname_orgid_d.keys())
match_orgid_set.remove(None)

for i in range(len(initial_domain_tld_plus_one_l)):
    if initial_domain_tld_plus_one_l[i] in match_orgid_set:
        org_id_first_party_l[i] = True
        
print(org_id_first_party_l.count(None) + org_id_first_party_l.count(True) == bright_data_df.shape[0])
print(f"{round(org_id_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% is the first party based on Org ID.")

True
99.94076% is the first party based on Org ID.


In [156]:
bright_data_df["org_id_first_party"] = org_id_first_party_l

In [157]:
bright_data_df.iloc[50:60]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party,as_number_first_party,org_id_first_party
50,hr,61094,20220804-HR-61094-httpswww.tportal.hr-DNS_Requ...,www.tportal.hr,tportal.hr,tportal.hr,True,True,True,True
51,hr,61094,20220804-HR-61094-httpswww.poslovni.hr-DNS_Req...,hr.hit.gemius.pl,poslovni.hr,gemius.pl,None,True,True,True
52,hr,61094,20220804-HR-61094-httpswww.net.hr-DNS_Requests...,s.amazon-adsystem.com,net.hr,adsystem.com,None,True,True,True
53,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,www.youtube.com,gloria.hr,youtube.com,None,True,True,True
54,hr,61094,20220804-HR-61094-httpswww.index.hr-DNS_Reques...,script.dotmetrics.net,index.hr,dotmetrics.net,None,True,True,True
55,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,connect.facebook.net,vecernji.hr,facebook.net,None,True,True,True
56,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,cdnjs.cloudflare.com,vecernji.hr,cloudflare.com,None,True,True,True
57,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-beacon.s-onetag.com,jutarnji.hr,onetag.com,None,True,True,True
58,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,apps.jutarnji.hr,gloria.hr,jutarnji.hr,None,True,True,True
59,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-metrics-collector-beta.s-onetag.com,jutarnji.hr,onetag.com,None,True,True,True


In [158]:
bright_data_df.shape

(1075319, 10)

## 6.5. org_name_first_party_l

In [159]:
bright_data_10col_df = copy.deepcopy(bright_data_df)

In [160]:
org_name_first_party_l = bright_data_df.shape[0] * [None]
match_orgid_set = set(match_hostname_orgname_d.keys())
match_orgid_set.remove(None)

for i in range(len(initial_domain_tld_plus_one_l)):
    if initial_domain_tld_plus_one_l[i] in match_orgid_set:
        org_name_first_party_l[i] = True
        
print(org_name_first_party_l.count(None) + org_name_first_party_l.count(True) == bright_data_df.shape[0])
print(f"{round(org_name_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% are the first party based on Org Name.")

True
99.94076% is the first party based on Org ID.


In [161]:
bright_data_df["org_name_first_party"] = org_name_first_party_l

In [162]:
bright_data_df.iloc[50:60]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party,as_number_first_party,org_id_first_party,org_name_first_party
50,hr,61094,20220804-HR-61094-httpswww.tportal.hr-DNS_Requ...,www.tportal.hr,tportal.hr,tportal.hr,True,True,True,True,True
51,hr,61094,20220804-HR-61094-httpswww.poslovni.hr-DNS_Req...,hr.hit.gemius.pl,poslovni.hr,gemius.pl,None,True,True,True,True
52,hr,61094,20220804-HR-61094-httpswww.net.hr-DNS_Requests...,s.amazon-adsystem.com,net.hr,adsystem.com,None,True,True,True,True
53,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,www.youtube.com,gloria.hr,youtube.com,None,True,True,True,True
54,hr,61094,20220804-HR-61094-httpswww.index.hr-DNS_Reques...,script.dotmetrics.net,index.hr,dotmetrics.net,None,True,True,True,True
55,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,connect.facebook.net,vecernji.hr,facebook.net,None,True,True,True,True
56,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,cdnjs.cloudflare.com,vecernji.hr,cloudflare.com,None,True,True,True,True
57,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-beacon.s-onetag.com,jutarnji.hr,onetag.com,None,True,True,True,True
58,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,apps.jutarnji.hr,gloria.hr,jutarnji.hr,None,True,True,True,True
59,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-metrics-collector-beta.s-onetag.com,jutarnji.hr,onetag.com,None,True,True,True,True


In [163]:
bright_data_df.shape

(1075319, 11)

## 6.6. Altogether

In [164]:
bright_data_11col_df = copy.deepcopy(bright_data_df)

In [196]:
bright_data_df['first_party'] = np.select([bright_data_df.iloc[:, 6:10].any(axis="columns")], [True], default=None)

In [208]:
print(f"{round(list(bright_data_df['first_party']).count(True) / bright_data_df.shape[0] * 100, 5)}% is the first party overall.")

99.94076% is the first party overall.


In [201]:
bright_data_df.iloc[20:30]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party,as_number_first_party,org_id_first_party,org_name_first_party,first_party
20,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,www.google.hr,None,None,None,None,None,None,None,None
21,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,apis.google.com,None,None,None,None,None,None,None,None
22,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,ogs.google.hr,None,None,None,None,None,None,None,None
23,hr,61094,20220804-HR-61094-httpswww.njuskalo.hr-DNS_Req...,ls.hit.gemius.pl,njuskalo.hr,gemius.pl,None,True,True,True,True,True
24,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,aax-eu.amazon-adsystem.com,24sata.hr,adsystem.com,None,True,True,True,True,True
25,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,buy.tinypass.com,24sata.hr,tinypass.com,None,True,True,True,True,True
26,hr,61094,20220804-HR-61094-httpswww.roblox.com-DNS_Requ...,static.rbxcdn.com,roblox.com,rbxcdn.com,None,True,True,True,True,True
27,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,cdn.ampproject.org,gloria.hr,ampproject.org,None,True,True,True,True,True
28,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,ads.yahoo.com,24sata.hr,yahoo.com,None,True,True,True,True,True
29,hr,61094,20220804-HR-61094-httpswww.srce.hr-DNS_Request...,www.srce.unizg.hr,srce.hr,unizg.hr,None,True,True,True,True,True


In [212]:
bright_data_12col_df = copy.deepcopy(bright_data_df)

In [ ]:
os.chdir(output_path_s)
print(os.getcwd())
bright_data_df.to_csv("domains_bright_data_11col.csv")

# 7. Parser list for trackers

## 7.1. Read TXT: easyprivacy.txt

In [213]:
os.chdir(input_path_s)
print(os.getcwd())

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain/input


In [214]:
easyprivacy_df = pd.read_csv("easyprivacy.txt", on_bad_lines="skip")

In [215]:
easyprivacy_l = easyprivacy_df["[Adblock Plus 1.1]"].values.tolist()

In [222]:
parser_l = [parser for parser in easyprivacy_l if parser[0] != "!"]

## 7.2. Parse

In [232]:
start_time = time.time()
is_target_domain_tracker_l = []

for tld_plus_one in target_domain_tld_plus_one_l:
    is_tracker = False
    if tld_plus_one is not None:        
        for parser in parser_l:
            if tld_plus_one in parser:
                is_tracker = True
    is_target_domain_tracker_l.append(is_tracker)

elapsed_time = time.time() - start_time
print(str(timedelta(seconds=elapsed_time)))

0:29:00.977789


In [365]:
print(len(target_domain_tld_plus_one_l) == len(is_target_domain_tracker_l))
print(is_target_domain_tracker_l[20:30])
print(f"{round(np.count_nonzero(np.array(is_target_domain_tracker_l)) / len(is_target_domain_tracker_l) * 100, 5)}% of target domains are trackers.")

True
[False, False, False, True, False, True, False, True, True, False]
67.959% of target domains are trackers.


## 7.3. Add a column

In [240]:
bright_data_df["is_target_domain_tracker"] = is_target_domain_tracker_l

In [241]:
bright_data_df.shape

(1075319, 13)

In [244]:
bright_data_df.iloc[20:30]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party,as_number_first_party,org_id_first_party,org_name_first_party,first_party,is_target_domain_tracker
20,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,www.google.hr,None,None,None,None,None,None,None,None,False
21,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,apis.google.com,None,None,None,None,None,None,None,None,False
22,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,ogs.google.hr,None,None,None,None,None,None,None,None,False
23,hr,61094,20220804-HR-61094-httpswww.njuskalo.hr-DNS_Req...,ls.hit.gemius.pl,njuskalo.hr,gemius.pl,None,True,True,True,True,True,True
24,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,aax-eu.amazon-adsystem.com,24sata.hr,adsystem.com,None,True,True,True,True,True,False
25,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,buy.tinypass.com,24sata.hr,tinypass.com,None,True,True,True,True,True,True
26,hr,61094,20220804-HR-61094-httpswww.roblox.com-DNS_Requ...,static.rbxcdn.com,roblox.com,rbxcdn.com,None,True,True,True,True,True,False
27,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,cdn.ampproject.org,gloria.hr,ampproject.org,None,True,True,True,True,True,True
28,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,ads.yahoo.com,24sata.hr,yahoo.com,None,True,True,True,True,True,True
29,hr,61094,20220804-HR-61094-httpswww.srce.hr-DNS_Request...,www.srce.unizg.hr,srce.hr,unizg.hr,None,True,True,True,True,True,False


In [242]:
bright_data_13col_df = copy.deepcopy(bright_data_df)

In [247]:
os.chdir(output_path_s)
print(os.getcwd())
bright_data_df.to_csv("domains_bright_data_12col.csv")

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain/output


# 8. Functions for Google

In [250]:
def whois(ip_l):
    as_name_l = []
    as_number_l = []
    
    for ip in ip_l:
        cmd = f"whois -h whois.cymru.com {ip}"
        try:
            cymru_l = os.popen(cmd).read().split("\n")[1].split("|")

            for index, item in enumerate(cymru_l):
                if index == 0:
                    as_number_l.append(item.strip())
                if index == 2:
                    as_name_l.append(item.strip())
        except:
            print(f"Run whois again for this ip: {ip}")
                
    return as_name_l, as_number_l 

In [252]:
def get_descriptive_statistics_about_as_name_and_as_number(as_name_l, as_number_l):
    print(f"The length of AS Name is {len(as_name_l)}\n")
        
    print(f"The length of AS Number is {len(as_number_l)}\n")
    
    if len(as_name_l) == len(as_number_l):
        print("AS Name and AS Number have the same length.\n")
    elif len(as_name_l) > len(as_number_l):
        print("AS Name has a longer length than AS Number.\n")
    else:
        print("AS Number has a longer length than AS Name.\n")
   
    if len(as_name_l) == len(set(as_name_l)):
        print("No duplicates in AS Name.\n")
    else:
        print(f"Duplicates in AS Name. set(as_name_l) = {set(as_name_l)}\n")

    if len(as_number_l) == len(set(as_number_l)):
        print("No duplicates in AS Number.\n")
    else:
        print(f"Duplicates in AS Number. set(as_number_l) = {set(as_number_l)}\n")

# 9. Google.com

In [248]:
os.chdir(input_path_s)
print(os.getcwd())

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain/input


## 9.1. dig: hostname to ip

In [258]:
google_ip_l = []

hostname_s = "google.com"

cmd = "dig +short " + hostname_s
ip_ll = os.popen(cmd).read().split("\n")

for index, ip in enumerate(ip_ll):
    if ip != "":
        google_ip_l.append(ip)
    
print(google_ip_l)

['142.250.188.238']


## 9.2. whois: ip to AS Number and AS Name

In [251]:
google_as_name_l, google_as_number_l = whois(google_ip_l)

In [253]:
get_descriptive_statistics_about_as_name_and_as_number(google_as_name_l, google_as_number_l)

The length of AS Name is 1

The length of AS Number is 1

AS Name and AS Number have the same length.

No duplicates in AS Name.

No duplicates in AS Number.



In [259]:
google_as_name_set, google_as_number_set = set(google_as_name_l), set(google_as_number_l)

## 9.3. AS Number to AS2Org_OrgID

In [265]:
# asn_orgid_d
# orgid_orgname_d

In [285]:
google_orgname_l = []

for as_numbrer in google_as_number_l:
    google_orgname_l.append(asn_orgid_d[int(as_numbrer)])

print(google_orgname_l)

['GOGL-ARIN']


In [287]:
google_orgname_set = set(google_orgname_l)

## 9.4. Summary

In [288]:
print(google_ip_l)
print(google_as_name_set)
print(google_as_number_set)
print(google_orgname_set)

['142.250.188.238']
{'GOOGLE, US'}
{'15169'}
{'GOGL-ARIN'}


## 9.5. Convert a set into a string, since a set has only 1 elements

In [293]:
for google_as_name in google_as_name_set:
    google_as_name_s = google_as_name

In [294]:
for google_as_number in google_as_number_set:
    google_as_number_s = google_as_number

In [295]:
for google_orgname in google_orgname_set:
    google_orgname_s = google_orgname

In [296]:
print(google_as_name_s)
print(google_as_number_s)
print(google_orgname_s)

GOOGLE, US
15169
GOGL-ARIN


# 10. Google Match

## 10.1. Include asn == "google". The initial status.

In [349]:
is_target_domain_google_l = len(target_domain_tld_plus_one_l) * [False]
print(len(is_target_domain_google_l) == bright_data_df.shape[0])

True


In [350]:
index_not_to_check_set = set()

for i in range(len(is_target_domain_google_l)):
    if bright_data_df["asn"][i] == "google":
        is_target_domain_google_l[i] = True
        index_not_to_check_set.add(i)

print(is_target_domain_google_l.count(True) == len(index_not_to_check_set))
print(f"{round(is_target_domain_google_l.count(True) / len(is_target_domain_google_l) * 100, 5)}% of target domain is Google. This is what the initial dataset is given to me.")

True
0.05924% of target domain is Google. This is what the initial dataset is given to me.


## 10.2. Method: AS (AS Name)

In [351]:
for hostname, as_name_l in target_domain_hostname_as_d.items():
    if google_as_name_s in as_name_l:
        for index, tld_plus_one in enumerate(target_domain_tld_plus_one_l):
            if tld_plus_one == hostname and index not in index_not_to_check_set:
                is_target_domain_google_l[index] = True
                index_not_to_check_set.add(index)

print(is_target_domain_google_l.count(True) == len(index_not_to_check_set))
print(f"{round(is_target_domain_google_l.count(True) / len(is_target_domain_google_l) * 100, 5)}% of target domain is Google.")

True
21.17139% of target domain is Google.


## 10.3. Method: ASN (AS Number)

In [352]:
for hostname, as_number_l in target_domain_hostname_asn_d.items():
    if google_as_name_s in as_number_l:
        for index, tld_plus_one in enumerate(target_domain_tld_plus_one_l):
            if tld_plus_one == hostname and index not in index_not_to_check_set:
                is_target_domain_google_l[index] = True
                index_not_to_check_set.add(index)

print(is_target_domain_google_l.count(True) == len(index_not_to_check_set))
print(f"{round(is_target_domain_google_l.count(True) / len(is_target_domain_google_l) * 100, 5)}% of target domain is Google.")

True
21.17139% of target domain is Google.


## 10.4.

## 10.5. Method: Org Name

In [353]:
for hostname, orgname_l in target_domain_hostname_orgname_d.items():
    if google_as_name_s in orgname_l:
        for index, tld_plus_one in enumerate(target_domain_tld_plus_one_l):
            if tld_plus_one == hostname and index not in index_not_to_check_set:
                is_target_domain_google_l[index] = True
                index_not_to_check_set.add(index)

print(is_target_domain_google_l.count(True) == len(index_not_to_check_set))
print(f"{round(is_target_domain_google_l.count(True) / len(is_target_domain_google_l) * 100, 5)}% of target domain is Google.")

True
21.17139% of target domain is Google.


In [354]:
is_target_domain_google_l[20:30]

[True, True, True, False, False, False, False, True, False, False]

## 10.6. Add a column

In [355]:
bright_data_df["is_target_domain_google"] = is_target_domain_google_l

In [356]:
bright_data_df.shape

(1075319, 14)

In [359]:
bright_data_df.iloc[20:30]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party,as_number_first_party,org_id_first_party,org_name_first_party,first_party,is_target_domain_tracker,is_target_domain_google
20,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,www.google.hr,None,None,None,None,None,None,None,None,False,True
21,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,apis.google.com,None,None,None,None,None,None,None,None,False,True
22,hr,google,20220804-HR-google-httpswww.google.hr-DNS_Requ...,ogs.google.hr,None,None,None,None,None,None,None,None,False,True
23,hr,61094,20220804-HR-61094-httpswww.njuskalo.hr-DNS_Req...,ls.hit.gemius.pl,njuskalo.hr,gemius.pl,None,True,True,True,True,True,True,False
24,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,aax-eu.amazon-adsystem.com,24sata.hr,adsystem.com,None,True,True,True,True,True,False,False
25,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,buy.tinypass.com,24sata.hr,tinypass.com,None,True,True,True,True,True,True,False
26,hr,61094,20220804-HR-61094-httpswww.roblox.com-DNS_Requ...,static.rbxcdn.com,roblox.com,rbxcdn.com,None,True,True,True,True,True,False,False
27,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,cdn.ampproject.org,gloria.hr,ampproject.org,None,True,True,True,True,True,True,True
28,hr,61094,20220804-HR-61094-httpswww.24sata.hr-DNS_Reque...,ads.yahoo.com,24sata.hr,yahoo.com,None,True,True,True,True,True,True,False
29,hr,61094,20220804-HR-61094-httpswww.srce.hr-DNS_Request...,www.srce.unizg.hr,srce.hr,unizg.hr,None,True,True,True,True,True,False,False


In [360]:
bright_data_14col_df = copy.deepcopy(bright_data_df)

In [361]:
os.chdir(output_path_s)
print(os.getcwd())
bright_data_df.to_csv("domains_bright_data_14col.csv")

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain/output


# 11. Important Statistics

In [368]:
print(f"{round(as_name_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% are the first party based on AS.")
print(f"{round(as_number_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% are the first party based on ASN.")
print(f"{round(org_id_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% are the first party based on Org ID.")
print(f"{round(org_name_first_party_l.count(True) / bright_data_df.shape[0] * 100, 5)}% are the first party based on Org Name.")

98.06457% are the first party based on AS.
98.06457% are the first party based on ASN.
99.94076% are the first party based on Org ID.
99.94076% are the first party based on Org Name.


In [364]:
print(f"{round(np.count_nonzero(np.array(is_target_domain_tracker_l)) / len(is_target_domain_tracker_l) * 100, 5)}% of target domain are trackers.")
print(f"{round(is_target_domain_google_l.count(True) / len(is_target_domain_google_l) * 100, 5)}% of target domain are Google.")

67.959% of target domain are trackers.
21.17139% of target domain are Google.
